In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import pickle as pickle

import sys
path = '../../scripts/'
sys.path.insert(0,path)
from RipserToDict import ripser_to_dict
from PlotPersistence import plot_persistence

In [4]:
with open('params.pickle', 'rb') as f:
    params = pickle.load(f)

## Functions

In [9]:
def get_label(string,params):
    shapes = params['shapes']
    label = 0
    for name in shapes['names']:
        if name == string:
            break
        else:
            label += 1
    return label

In [21]:
def create_circle_upper_distance_matrix(params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('circle', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    seed = shapes['seed']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # construct unit circle, r=1±dev, 0<theta<2*pi
    r = 1. - dev / 2. + dev * np.random.rand(n_points)
    theta = 2. * np.pi * np.random.rand(n_points)
    
    # transform into Euclidean
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    # calculate distances
    upper_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            upper_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2)
            
    return upper_distance_matrix

In [27]:
def create_sphere_upper_distance_matrix(params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('sphere', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    seed = shapes['seed']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # construct unit sphere in spherical coordinates: r=1±dev, 0<theta<pi, 0<phi<2*pi
    r = 1. - dev / 2. + dev * np.random.rand(n_points)
    theta = np.pi * np.random.rand(n_points)
    phi = 2. * np.pi * np.random.rand(n_points)
    
    # transform into Euclidean
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    
    # calculate distances
    upper_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            upper_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2 + (z[j] - [z[i]])**2)
            
    return upper_distance_matrix

In [29]:
def create_torus_upper_distance_matrix(params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('torus', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    seed = shapes['seed']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # create noise
    A = 1. - dev / 2. +  dev * np.random.rand(n_points)
    
    # construct 0<theta<2*pi, 0<phi<2*pi
    theta = 2. * np.pi * np.random.rand(n_points)
    phi = 2. * np.pi * np.random.rand(n_points)
    
    # construct torus in R4
    x = A * np.cos(theta)
    y = A * np.sin(theta)
    z = A * np.cos(phi)
    t = A * np.sin(phi)
    
    # calculate distances
    upper_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            upper_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2\
                                                 + (z[j] - [z[i]])**2 + (t[j] - t[i])**2)
            
    return upper_distance_matrix

## Run

In [ ]:
for shape in params['shapes']['names']:
    for seed in params['all_seeds']:
        if shape == 'circle':
            distance_matrix = create_circle_distance_matrix(params)
        elif shape == 'sphere':
            distance_matrix = create_sphere_upper_distance_matrix(params)
        elif shape == 'torus':
            distance_matrix = create_torus_upper_distance_matrix(params)
        np.savetxt(params['paths']['code_to_data'] + 'temp/distance.dat', distance_matrix, delimiter=',')

        !cd {params['paths']['code_to_ripser']}; \
        ./ripser --format upper-distance --dim {params['ripser']['dim']} --threshold {params['ripser']['threshold']} \
        {params['paths']['ripser_to_data']}temp/distance.dat \
        > {params['paths']['ripser_to_data']}temp/persistence.txt

        persistence = ripser_to_dict(params['paths']['code_to_data'] + 'temp/persistence.txt', params).copy()

        with open(params['paths']['code_to_data'] + 'persistences/' + shape + '/' + str(seed) + '.txt', "wb") as f:
            pickle.dump(persistence, f)

In [30]:
print(params['shapes']['n_points'])
params['shapes']['seed'] = 1
label = create_torus_upper_distance_matrix(params)
print(label)

200
[[ 0.          2.12409154  0.49185659 ...,  2.2242119   2.01114308
   2.16622608]
 [ 2.12409154  0.          2.20428777 ...,  2.25224552  1.34116528  0.        ]
 [ 0.49185659  2.20428777  0.         ...,  1.95025223  0.          0.        ]
 ..., 
 [ 2.2242119   2.25224552  1.95025223 ...,  0.          0.          0.        ]
 [ 2.01114308  1.34116528  0.         ...,  0.          0.          0.        ]
 [ 2.16622608  0.          0.         ...,  0.          0.          0.        ]]


In [24]:
print(np.random.rand(3))

[ 0.95943432  0.80396089  0.03232307]


In [31]:
a = 'ls'
!{a}

create_shapes_and_their_PDs.ipynb params.mat
params.ipynb                      params.pickle
